In [110]:
from pymongo import MongoClient
from pymongo import WriteConcern
from pymongo import ReadPreference
from pymongo.read_concern import ReadConcern

import time


#### Task 1

Set up replication with following config: Primary with Two Secondary Members.

*Replica set configured with Docker containers. Claster running and initialization implemented with docker-compose.*

Files:
- docker-compose.yml
- rs0-init.sh


#### Initializing connection to replica set

In [3]:
connection = MongoClient('mongodb://mongodb_node_1,mongodb_node_2,mongodb_node_3/', replicaSet='rs0')
print(connection)
print(connection.nodes)
time.sleep(7)
print(connection.nodes)

MongoClient(host=['mongodb_node_3:27017', 'mongodb_node_2:27017', 'mongodb_node_1:27017'], document_class=dict, tz_aware=False, connect=True, replicaset='rs0')
frozenset()
frozenset({('mongodb_node_3', 27017), ('mongodb_node_2', 27017), ('mongodb_node_1', 27017)})


In [4]:
print(connection.nodes)

frozenset({('mongodb_node_3', 27017), ('mongodb_node_2', 27017), ('mongodb_node_1', 27017)})



#### Task 2

Demonstrate data writes with different wright conserns:

In [7]:
db = connection.inventory

In [17]:
db.items.drop()

In [194]:
items = [
    { '_id': 1, 'item': 'Ipnone 11', 'memory': '64Gb', 'price': 18499, 'producer': 'Apple', 'category': 'Smartphones' },
    { '_id': 2, 'item': 'Ipnone 13', 'price': 29999, 'producer': 'Apple', 'category': 'Smartphones' },
    { '_id': 3, 'item': 'Ipnone 13 Pro', 'price': 37499, 'producer': 'Apple', 'category': 'Smartphones' },
    { '_id': 4, 'item': 'Ipnone 13 Pro Max', 'memory': '128Gb', 'price': 41499, 'producer': 'Apple', 'category': 'Smartphones' },
    { '_id': 5, 'item': 'Galaxy S21 FE', 'memory': '64Gb', 'price': 21999, 'producer': 'Samsung', 'category': 'Smartphones' },
    { '_id': 6, 'item': 'Galaxy S21', 'memory': '128Gb', 'price': 22999, 'producer': 'Samsung', 'category': 'Smartphones' },
    { '_id': 7, 'item': 'Galaxy S21 Plus', 'price': 23999, 'producer': 'Samsung', 'category': 'Smartphones' },
    { '_id': 8, 'item': 'Galaxy S21 Ultra', 'price': 39999, 'producer': 'Samsung', 'category': 'Smartphones' },
    { '_id': 9, 'item': 'Apple Watch Series 7', 'connectivity': ['WiFi', 'Bluetooth'], 'price': 13999, 'type': 'smartwatch', 'producer': 'Samsung', 'category': 'Wearables' },
    { '_id': 10, 'item': 'Galaxy Watch 4 Classic', 'connectivity': ['WiFi', 'Bluetooth', 'LTE'], 'price': 11499, 'type': 'smartwatch', 'producer': 'Apple', 'category': 'Wearables' },
    { '_id': 11, 'item': 'Galaxy Tab S7',  'memory': '128Gb', 'display': '11.4"', 'price': 23999, 'producer': 'Samsung', 'category': 'Tablets' },
    { '_id': 12, 'item': 'iPad Pro',  'memory': '128Gb', 'display': '12.9"', 'price': 41999, 'producer': 'Apple', 'category': 'Tablets' },
    { '_id': 13, 'item': 'Samsung QE55', 'technology': 'QLED', 'price': 30999, 'producer': 'Samsung', 'category': 'TV' },
    { '_id': 14, 'item': 'Hisense 55E', 'technology': 'IPS', 'price': 19999, 'producer': 'Hisense', 'category': 'TV' },
    { '_id': 15, 'item': 'LG OLED55', 'technology': 'OLED', 'price': 41999, 'producer': 'LG', 'category': 'TV' },
    { '_id': 16, 'item': 'Philips 55OLED', 'technology': 'OLED', 'price': 45999, 'producer': 'Philips', 'category': 'TV' },
]

##### Unacknowledged

In [19]:
item = items[0]

id = db.items.insert_one(item, WriteConcern(w=0)).inserted_id
id

1

##### Acknowledged

In [20]:
item = items[1]

id = db.items.insert_one(item, WriteConcern(w=1)).inserted_id
id

2

##### Journaled

In [21]:
item = items[2]

id = db.items.insert_one(item, WriteConcern(w=1, j=True)).inserted_id
id

3

##### AcknowledgedReplica

In [22]:
item = items[3]

id = db.items.insert_one(item, WriteConcern(w='majority', wtimeout=5000)).inserted_id
id

4

#### Task 3

Demostrate Read Preference Modes

##### primary

In [33]:
response = db.items.with_options(read_preference=ReadPreference.PRIMARY).find({})
print(list(response))

[{'_id': 1, 'item': 'Ipnone 11', 'memory': '64Gb', 'price': 18499, 'producer': 'Apple', 'category': 'Smartphones'}, {'_id': 2, 'item': 'Ipnone 13', 'price': 29999, 'producer': 'Apple', 'category': 'Smartphones'}, {'_id': 3, 'item': 'Ipnone 13 Pro', 'price': 37499, 'producer': 'Apple', 'category': 'Smartphones'}, {'_id': 4, 'item': 'Ipnone 13 Pro Max', 'memory': '128Gb', 'price': 41499, 'producer': 'Apple', 'category': 'Smartphones'}]


##### primaryPreferred

In [36]:
response = db.items.with_options(read_preference=ReadPreference.PRIMARY_PREFERRED).find({})
print(list(response))

[{'_id': 1, 'item': 'Ipnone 11', 'memory': '64Gb', 'price': 18499, 'producer': 'Apple', 'category': 'Smartphones'}, {'_id': 2, 'item': 'Ipnone 13', 'price': 29999, 'producer': 'Apple', 'category': 'Smartphones'}, {'_id': 3, 'item': 'Ipnone 13 Pro', 'price': 37499, 'producer': 'Apple', 'category': 'Smartphones'}, {'_id': 4, 'item': 'Ipnone 13 Pro Max', 'memory': '128Gb', 'price': 41499, 'producer': 'Apple', 'category': 'Smartphones'}]


##### secondary

In [37]:
response = db.items.with_options(read_preference=ReadPreference.SECONDARY).find({})
print(list(response))

[{'_id': 1, 'item': 'Ipnone 11', 'memory': '64Gb', 'price': 18499, 'producer': 'Apple', 'category': 'Smartphones'}, {'_id': 2, 'item': 'Ipnone 13', 'price': 29999, 'producer': 'Apple', 'category': 'Smartphones'}, {'_id': 3, 'item': 'Ipnone 13 Pro', 'price': 37499, 'producer': 'Apple', 'category': 'Smartphones'}, {'_id': 4, 'item': 'Ipnone 13 Pro Max', 'memory': '128Gb', 'price': 41499, 'producer': 'Apple', 'category': 'Smartphones'}]


##### secondaryPreferred

In [40]:
response = db.items.with_options(read_preference=ReadPreference.SECONDARY_PREFERRED).find({})
print(list(response))

[{'_id': 1, 'item': 'Ipnone 11', 'memory': '64Gb', 'price': 18499, 'producer': 'Apple', 'category': 'Smartphones'}, {'_id': 2, 'item': 'Ipnone 13', 'price': 29999, 'producer': 'Apple', 'category': 'Smartphones'}, {'_id': 3, 'item': 'Ipnone 13 Pro', 'price': 37499, 'producer': 'Apple', 'category': 'Smartphones'}, {'_id': 4, 'item': 'Ipnone 13 Pro Max', 'memory': '128Gb', 'price': 41499, 'producer': 'Apple', 'category': 'Smartphones'}]


##### nearest

In [41]:
response = db.items.with_options(read_preference=ReadPreference.NEAREST).find({})
print(list(response))

[{'_id': 1, 'item': 'Ipnone 11', 'memory': '64Gb', 'price': 18499, 'producer': 'Apple', 'category': 'Smartphones'}, {'_id': 2, 'item': 'Ipnone 13', 'price': 29999, 'producer': 'Apple', 'category': 'Smartphones'}, {'_id': 3, 'item': 'Ipnone 13 Pro', 'price': 37499, 'producer': 'Apple', 'category': 'Smartphones'}, {'_id': 4, 'item': 'Ipnone 13 Pro Max', 'memory': '128Gb', 'price': 41499, 'producer': 'Apple', 'category': 'Smartphones'}]


#### Task 4

Shut down one secondary node. Write a document with write_conser = 3 and limitless timeout. During timeout run switched off secondary node.

In [89]:
members = connection.admin.command('replSetGetStatus')['members']
members_status = [{'_id': member['_id'], 'name': member['name'], 'health': member['health'], 'stateStr': member['stateStr']}for member in members]
print(*members_status, sep='\n')

{'_id': 1, 'name': 'mongodb_node_1:27017', 'health': 1.0, 'stateStr': 'PRIMARY'}
{'_id': 2, 'name': 'mongodb_node_2:27017', 'health': 0.0, 'stateStr': '(not reachable/healthy)'}
{'_id': 3, 'name': 'mongodb_node_3:27017', 'health': 1.0, 'stateStr': 'SECONDARY'}


In [92]:
item = items[4]

id = db.items.with_options(write_concern=WriteConcern(w=3)).insert_one(item).inserted_id
id

5

In [94]:
members = connection.admin.command('replSetGetStatus')['members']
members_status = [{'_id': member['_id'], 'name': member['name'], 'health': member['health'], 'stateStr': member['stateStr']}for member in members]
print(*members_status, sep='\n')

{'_id': 1, 'name': 'mongodb_node_1:27017', 'health': 1.0, 'stateStr': 'PRIMARY'}
{'_id': 2, 'name': 'mongodb_node_2:27017', 'health': 1.0, 'stateStr': 'SECONDARY'}
{'_id': 3, 'name': 'mongodb_node_3:27017', 'health': 1.0, 'stateStr': 'SECONDARY'}


#### Task 5

With analogy to prevoius task, but with timeout set and ends. Check if document is writen and if it could be read with readConsern: 'majority'


In [103]:
members = connection.admin.command('replSetGetStatus')['members']
members_status = [{'_id': member['_id'], 'name': member['name'], 'health': member['health'], 'stateStr': member['stateStr']}for member in members]
print(*members_status, sep='\n')

{'_id': 1, 'name': 'mongodb_node_1:27017', 'health': 1.0, 'stateStr': 'PRIMARY'}
{'_id': 2, 'name': 'mongodb_node_2:27017', 'health': 0.0, 'stateStr': '(not reachable/healthy)'}
{'_id': 3, 'name': 'mongodb_node_3:27017', 'health': 1.0, 'stateStr': 'SECONDARY'}


In [105]:
item = items[5]

id = db.items.with_options(write_concern=WriteConcern(w=3, wtimeout=5000)).insert_one(item).inserted_id
id

WTimeoutError: waiting for replication timed out, full error: {'code': 64, 'codeName': 'WriteConcernFailed', 'errmsg': 'waiting for replication timed out', 'errInfo': {'wtimeout': True, 'writeConcern': {'w': 3, 'wtimeout': 5000, 'provenance': 'clientSupplied'}}}

In [113]:
response = db.items.with_options(read_concern=ReadConcern(level='majority')).find({'_id': 6})
print(list(response))

[{'_id': 6, 'item': 'Galaxy S21', 'memory': '128Gb', 'price': 22999, 'producer': 'Samsung', 'category': 'Smartphones'}]


#### Task 6

Demonstrate Primary node reelection by switching off current Primary node, and confirm replication of new data after old primary is back to the replica set.

In [173]:
members = connection.admin.command('replSetGetStatus')['members']
members_status = [{'_id': member['_id'], 'name': member['name'], 'health': member['health'], 'stateStr': member['stateStr']}for member in members]
print(*members_status, sep='\n')

{'_id': 1, 'name': 'mongodb_node_1:27017', 'health': 1.0, 'stateStr': 'PRIMARY'}
{'_id': 2, 'name': 'mongodb_node_2:27017', 'health': 1.0, 'stateStr': 'SECONDARY'}
{'_id': 3, 'name': 'mongodb_node_3:27017', 'health': 1.0, 'stateStr': 'SECONDARY'}


In [181]:
members = connection.admin.command('replSetGetStatus')['members']
members_status = [{'_id': member['_id'], 'name': member['name'], 'health': member['health'], 'stateStr': member['stateStr']}for member in members]
print(*members_status, sep='\n')

{'_id': 1, 'name': 'mongodb_node_1:27017', 'health': 0.0, 'stateStr': '(not reachable/healthy)'}
{'_id': 2, 'name': 'mongodb_node_2:27017', 'health': 1.0, 'stateStr': 'PRIMARY'}
{'_id': 3, 'name': 'mongodb_node_3:27017', 'health': 1.0, 'stateStr': 'SECONDARY'}


In [182]:
items_to_insert = items[-4:]

ids = db.items.with_options(write_concern=WriteConcern(w=1)).insert_many(items_to_insert).inserted_ids
ids

[13, 14, 15, 16]

In [161]:
db.items.delete_one({'_id': 16})

In [183]:
response = db.items.with_options(read_concern=ReadConcern(level='majority')).find({'category': 'TV'})
print(*list(response), sep='\n')

{'_id': 13, 'item': 'Samsung QE55', 'technology': 'QLED', 'price': 30999, 'producer': 'Samsung', 'category': 'TV'}
{'_id': 14, 'item': 'Hisense 55E', 'technology': 'IPS', 'price': 19999, 'producer': 'Hisense', 'category': 'TV'}
{'_id': 15, 'item': 'LG OLED55', 'technology': 'OLED', 'price': 41999, 'producer': 'LG', 'category': 'TV'}
{'_id': 16, 'item': 'Philips 55OLED', 'technology': 'OLED', 'price': 45999, 'producer': 'Philips', 'category': 'TV'}


In [187]:
members = connection.admin.command('replSetGetStatus')['members']
members_status = [{'_id': member['_id'], 'name': member['name'], 'health': member['health'], 'stateStr': member['stateStr']}for member in members]
print(*members_status, sep='\n')

{'_id': 1, 'name': 'mongodb_node_1:27017', 'health': 1.0, 'stateStr': 'SECONDARY'}
{'_id': 2, 'name': 'mongodb_node_2:27017', 'health': 1.0, 'stateStr': 'PRIMARY'}
{'_id': 3, 'name': 'mongodb_node_3:27017', 'health': 1.0, 'stateStr': 'SECONDARY'}


In [188]:
members = connection.admin.command('replSetGetStatus')['members']
members_status = [{'_id': member['_id'], 'name': member['name'], 'health': member['health'], 'stateStr': member['stateStr']}for member in members]
print(*members_status, sep='\n')

{'_id': 1, 'name': 'mongodb_node_1:27017', 'health': 1.0, 'stateStr': 'SECONDARY'}
{'_id': 2, 'name': 'mongodb_node_2:27017', 'health': 1.0, 'stateStr': 'PRIMARY'}
{'_id': 3, 'name': 'mongodb_node_3:27017', 'health': 0.0, 'stateStr': '(not reachable/healthy)'}


In [189]:
response = db.items.with_options(read_preference=ReadPreference.SECONDARY).find({'category': 'TV'})
print(*list(response), sep='\n')

{'_id': 14, 'item': 'Hisense 55E', 'technology': 'IPS', 'price': 19999, 'producer': 'Hisense', 'category': 'TV'}
{'_id': 15, 'item': 'LG OLED55', 'technology': 'OLED', 'price': 41999, 'producer': 'LG', 'category': 'TV'}
{'_id': 13, 'item': 'Samsung QE55', 'technology': 'QLED', 'price': 30999, 'producer': 'Samsung', 'category': 'TV'}
{'_id': 16, 'item': 'Philips 55OLED', 'technology': 'OLED', 'price': 45999, 'producer': 'Philips', 'category': 'TV'}


#### Task 7

Put Replica Set into inconsistent state using a delay of Primary node to detect the absence of secondary nodes. 

Switch off two secondary nodes. During 5 sec write a document on Primary node with write_consern=1. Check the document is writen.

Read writen values with different read_concerns - readConcern: {level: <"majority"|"local"| "linearizable">}

In [238]:
members = connection.admin.command('replSetGetStatus')['members']
members_status = [{'_id': member['_id'], 'name': member['name'], 'health': member['health'], 'stateStr': member['stateStr']}for member in members]
print(*members_status, sep='\n')

{'_id': 1, 'name': 'mongodb_node_1:27017', 'health': 1.0, 'stateStr': 'PRIMARY'}
{'_id': 2, 'name': 'mongodb_node_2:27017', 'health': 1.0, 'stateStr': 'SECONDARY'}
{'_id': 3, 'name': 'mongodb_node_3:27017', 'health': 1.0, 'stateStr': 'SECONDARY'}


In [234]:
db.items.delete_many({'category': 'Wearables'})

In [239]:
item = items[8]

ids = db.items.with_options(write_concern=WriteConcern(w=1)).insert_one(item).inserted_id
print(ids)

response = db.items.with_options(read_preference=ReadPreference.PRIMARY).find({'category': 'Wearables'})
print('readConcern: <default>')
print(*list(response), sep='\n')

response = db.items.with_options(read_concern=ReadConcern(level='local')).find({'category': 'Wearables'})
print('readConcern: "local"')
print(*list(response), sep='\n')

response = db.items.with_options(read_concern=ReadConcern(level='majority')).find({'category': 'Wearables'})
print('readConcern: "majority"')
print(*list(response), sep='\n')

response = db.items.with_options(read_concern=ReadConcern(level='linearizable')).find({'category': 'Wearables'})
print('readConcern: "linearizable"')
print(*list(response), sep='\n')

9
readConcern: <default>
{'_id': 9, 'item': 'Apple Watch Series 7', 'connectivity': ['WiFi', 'Bluetooth'], 'price': 13999, 'type': 'smartwatch', 'producer': 'Samsung', 'category': 'Wearables'}
readConcern: "local"
{'_id': 9, 'item': 'Apple Watch Series 7', 'connectivity': ['WiFi', 'Bluetooth'], 'price': 13999, 'type': 'smartwatch', 'producer': 'Samsung', 'category': 'Wearables'}
readConcern: "majority"

readConcern: "linearizable"


NotPrimaryError: not primary and secondaryOk=false, full error: {'topologyVersion': {'processId': ObjectId('62127de31a992e6acb3851b3'), 'counter': 12}, 'operationTime': Timestamp(1645379731, 2), 'ok': 0.0, 'errmsg': 'not primary and secondaryOk=false', 'code': 13435, 'codeName': 'NotPrimaryNoSecondaryOk', '$clusterTime': {'clusterTime': Timestamp(1645379731, 2), 'signature': {'hash': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00', 'keyId': 0}}}

Switch on two other nodes so that, they can't communicate with Primary node (could be switched off) and wait until elections of new Primary node is completed

In [241]:
members = connection.admin.command('replSetGetStatus')['members']
members_status = [{'_id': member['_id'], 'name': member['name'], 'health': member['health'], 'stateStr': member['stateStr']}for member in members]
print(*members_status, sep='\n')

{'_id': 1, 'name': 'mongodb_node_1:27017', 'health': 0.0, 'stateStr': '(not reachable/healthy)'}
{'_id': 2, 'name': 'mongodb_node_2:27017', 'health': 1.0, 'stateStr': 'SECONDARY'}
{'_id': 3, 'name': 'mongodb_node_3:27017', 'health': 1.0, 'stateStr': 'PRIMARY'}


Switch on old Primary node and demonstrate the availability of previously writen documents 

In [244]:
members = connection.admin.command('replSetGetStatus')['members']
members_status = [{'_id': member['_id'], 'name': member['name'], 'health': member['health'], 'stateStr': member['stateStr']}for member in members]
print(*members_status, sep='\n')

{'_id': 1, 'name': 'mongodb_node_1:27017', 'health': 1.0, 'stateStr': 'SECONDARY'}
{'_id': 2, 'name': 'mongodb_node_2:27017', 'health': 1.0, 'stateStr': 'SECONDARY'}
{'_id': 3, 'name': 'mongodb_node_3:27017', 'health': 1.0, 'stateStr': 'PRIMARY'}


In [247]:
response = db.items.find({'category': 'Wearables'})
print('Cheking if document available:')
print(*list(response), sep='\n')

Cheking if document available:



#### Task 8

Emulate eventual consistency by setting a delay for secondary node replication

    rs0:PRIMARY> cfg = rs.conf()
    {
            "_id" : "rs0",
            "version" : 1,
            "term" : 11,
            "protocolVersion" : NumberLong(1),
            "writeConcernMajorityJournalDefault" : true,
            "members" : [
                    {
                            "_id" : 1,     
                            "host" : "mongodb_node_1:27017",
                            "arbiterOnly" : false,
                            "buildIndexes" 
    : true,
                            "hidden" : false,
                            "priority" : 1,                        "tags" : {     

                            },
                            "slaveDelay" : 
    NumberLong(0),
                            "votes" : 1    
                    },
                    {
                            "_id" : 2,     
                            "host" : "mongodb_node_2:27017",
                            "arbiterOnly" : false,
                            "buildIndexes" 
    : true,
                            "hidden" : false,
                            "priority" : 1,                        "tags" : {     

                            },
                            "slaveDelay" : 
    NumberLong(0),
                            "votes" : 1    
                    },
                    {
                            "_id" : 3,     
                            "host" : "mongodb_node_3:27017",
                            "arbiterOnly" : false,
                            "buildIndexes" 
    : true,
                            "hidden" : false,
                            "priority" : 1,                        "tags" : {     

                            },
                            "slaveDelay" : 
    NumberLong(0),
                            "votes" : 1    
                    }
            ],
            "settings" : {
                    "chainingAllowed" : true,
                    "heartbeatIntervalMillis" : 2000,
                    "heartbeatTimeoutSecs" 
    : 10,
                    "electionTimeoutMillis" : 10000,
                    "catchUpTimeoutMillis" 
    : -1,
                    "catchUpTakeoverDelayMillis" : 30000,
                    "getLastErrorModes" : {
                    },
                    "getLastErrorDefaults" 
    : {
                            "w" : 1,       
                            "wtimeout" : 0 
                    },
                    "replicaSetId" : ObjectId("621245d3e64690eaa1330907")
            }
    }
    rs0:PRIMARY> cfg.members[0]
    {
            "_id" : 1,
            "host" : "mongodb_node_1:27017",
            "arbiterOnly" : false,
            "buildIndexes" : true,
            "hidden" : false,
            "priority" : 1,
            "tags" : {

            },
            "slaveDelay" : NumberLong(0),  
            "votes" : 1
    }
    rs0:PRIMARY> cfg.members[0].priority = 0
    0
    rs0:PRIMARY> cfg.members[0].hidden = true
    true
    rs0:PRIMARY> cfg.members[0].slaveDelay = 30
    30
    rs0:PRIMARY> rs.reconfig(cfg)
    {
            "ok" : 1,
            "$clusterTime" : {
                    "clusterTime" : Timestamp(1645387102, 1),
                    "signature" : {
                            "hash" : BinData(0,"AAAAAAAAAAAAAAAAAAAAAAAAAAA="),
                            "keyId" : NumberLong(0)
                    }
            },
            "operationTime" : Timestamp(1645387102, 1)
    }

In [291]:
members = connection.admin.command('replSetGetStatus')['members']
members_status = [
    {
        '_id': member['_id'], 
        'name': member['name'], 
        'health': member['health'], 
        'stateStr': member['stateStr'],
    } for member in members
]
print(*members_status, sep='\n')

{'_id': 1, 'name': 'mongodb_node_1:27017', 'health': 1.0, 'stateStr': 'SECONDARY'}
{'_id': 2, 'name': 'mongodb_node_2:27017', 'health': 0.0, 'stateStr': '(not reachable/healthy)'}
{'_id': 3, 'name': 'mongodb_node_3:27017', 'health': 1.0, 'stateStr': 'PRIMARY'}


#### Task 9

Leaving Primary and Secondary nodes with set up delay. Try to write several documents. Then try to read writen documents with readConcern: {level: "linearizable"}. There should be noticable latency until documents is replicating to the majority.

In [298]:
db.items.drop({})

In [299]:
start = time.time()
response = db.items.find({})
print(*list(response), sep='\n')
end = time.time()
print(f'Query took: {end-start} sec')


Query took: 0.005718231201171875 sec


In [300]:
id = db.items.with_options(write_concern=WriteConcern(w=0)).insert_one(items[6]).inserted_id
print(id)

id = db.items.with_options(write_concern=WriteConcern(w=0)).insert_one(items[7]).inserted_id
print(id)

id = db.items.with_options(write_concern=WriteConcern(w=0)).insert_one(items[8]).inserted_id
print(id)

id = db.items.with_options(write_concern=WriteConcern(w=0)).insert_one(items[9]).inserted_id
print(id)

ids = db.items.with_options(write_concern=WriteConcern(w=0)).insert_many(items[1:4]).inserted_ids
print(ids)


7
8
9
10
[2, 3, 4]


In [301]:
start = time.time()
response = db.items.with_options(read_concern=ReadConcern(level='linearizable')).find({'category': 'Smartphones'})
print(*list(response), sep='\n')
end = time.time()
print(f'Query took: {end-start} sec')

{'_id': 7, 'item': 'Galaxy S21 Plus', 'price': 23999, 'producer': 'Samsung', 'category': 'Smartphones'}
{'_id': 8, 'item': 'Galaxy S21 Ultra', 'price': 39999, 'producer': 'Samsung', 'category': 'Smartphones'}
{'_id': 2, 'item': 'Ipnone 13', 'price': 29999, 'producer': 'Apple', 'category': 'Smartphones'}
{'_id': 3, 'item': 'Ipnone 13 Pro', 'price': 37499, 'producer': 'Apple', 'category': 'Smartphones'}
{'_id': 4, 'item': 'Ipnone 13 Pro Max', 'memory': '128Gb', 'price': 41499, 'producer': 'Apple', 'category': 'Smartphones'}
Query took: 30.16016125679016 sec
